In [1]:
import gc
import sys
from statistics import mean
import time
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import itertools
from pathlib import Path
from scipy import special
import matplotlib.pyplot as plt
import datetime

import utils
import models
import learning_utils
from configurations import args_parser, arguments



In [2]:
start_time = time.time()
args = arguments()
#boardio is for the the tensorboardx prensation and textio is for written documentation
boardio, textio, best_val_acc, path_best_model, last_model_path = utils.initializations(args)
textio.cprint(str(args) if args.__class__.__name__ == 'Namespace' else str(vars(args)))

#mnist_train_data, mnist_test_loader  = utils.data(args)





{'exp_name': 'random selection', 'eval': False, 'data': 'mnist', 'norm_std': 0.5, 'norm_mean': 0.5, 'train_batch_size': 20, 'test_batch_size': 1000, 'model': 'cnn2', 'num_users': 30, 'num_users_per_round': 5, 'local_epochs': 1, 'local_iterations': 100, 'global_epochs': 200, 'tau_min': 0.05, 'privacy_noise': 'laplace', 'epsilon': 4, 'optimizer': 'sgd', 'lr': 0.01, 'momentum': 0.5, 'lr_scheduler': True, 'device': 'cpu', 'seed': 0, 'zeta_coeff': 1.5, 'alpha': 1, 'beta': 2, 'gamma': 1, 'max_seconds': 200, 'method_choosing_users': 'brute force', 'data_truncation': 1000, 'initiated_delay': 0.0}


In [3]:
# data
train_data, test_loader = utils.data(args)
#input in the CNNs is the number of channels and in linear models is the size of the flatten pictures
input, output, train_data, val_loader = utils.data_split(train_data, len(test_loader.dataset), args)

# model
if args.model == 'mlp':
    global_model = models.FC2Layer(input, output)
elif args.model == 'cnn2':
    global_model = models.CNN2Layer(input, output, args.data)
elif args.model == 'cnn3':
    if args.data == 'mnist':
        raise ValueError('CNN3 is not supported for MNIST')
    global_model = models.CNN3Layer()
elif args.model == 'cnn5':
    if args.data == 'mnist':
        raise ValueError('CNN3 is not supported for MNIST')
    global_model = models.CNN5Layer(input, output)
elif args.model == 'linear':
    global_model = models.Linear(input, output)



textio.cprint(str(summary(global_model)).encode('utf-8', errors='ignore').decode('utf-8', errors='ignore'))
global_model.to(args.device)

train_criterion = torch.nn.CrossEntropyLoss(reduction='mean')
test_criterion = torch.nn.CrossEntropyLoss(reduction='sum')

local_models = utils.federated_setup(global_model, train_data, args, i_i_d=True)
utils.update_data_equility_partititon(local_models, args)



Layer (type:depth-idx)                   Param #
CNN2Layer                                --
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            906
├─Linear: 1-3                            4,850
├─Linear: 1-4                            510
├─Dropout: 1-5                           --
├─Dropout: 1-6                           --
Total params: 6,422
Trainable params: 6,422
Non-trainable params: 0


In [4]:


choices_table = np.zeros((args.global_epochs, args.num_users))
num_of_obs_arr = np.zeros((1,args.num_users))
train_loss_list = []
val_acc_list = []
val_losses_list = []
global_epochs_time_list = []


time_counter = 0
for global_epoch in tqdm(range(1, args.global_epochs+1)):
    """Part 1: Choosing Users"""
    for usr_idx in range(args.num_users):
        local_models[usr_idx].update_g(global_epoch)
        local_models[usr_idx].update_ucb(global_epoch)
    
    print(f"iteration{global_epoch}")
    a=time.time()
    rounds_choise = utils.choose_users(local_models, args, method=args.method_choosing_users, privacy=False,
                                       initiated_delay=args.initiated_delay)
    #print(f"choose_users took {time.time()-a}")
    
    #choices_table[global_epoch-1, rounds_choise] = 1
    num_of_obs_arr[0,rounds_choise] += 1
    for usr_idx in rounds_choise:
        local_models[usr_idx].update_emp_avg()
        local_models[usr_idx].update_privacy_violation_and_reward()
        local_models[usr_idx].increase_num_of_obs()
        print(f"user {usr_idx}, g: {local_models[usr_idx].g},curr_delay = {local_models[usr_idx].last_access_time}, ucb: {local_models[usr_idx].ucb} num_of_obs: {local_models[usr_idx].num_of_obs}")
    
    max_delay = max([local_models[i].last_access_time for i in rounds_choise])
    #print(f"max_delay = {max_delay}")
    
    
    # """Part 2: Training"""
    # learning_utils.distribute_model(local_models, global_model)
    # users_avg_loss_over_local_epochs = []

    # for user_idx in rounds_choise:
    #     user_loss = []
    #     for local_epoch in range(args.local_epochs):
    #         user = local_models[user_idx]
    #         train_loss = learning_utils.train_one_epoch(user, train_criterion, args)
    #         if args.lr_scheduler:
    #             user.scheduler.step(train_loss)
    #         user_loss.append(train_loss)
    #     users_avg_loss_over_local_epochs.append(mean(user_loss))
    
    # avg_loss_over_chosen_users_curr_global_epoch = mean(users_avg_loss_over_local_epochs)
    # train_loss_list.append(avg_loss_over_chosen_users_curr_global_epoch)

    # #we return deltha_theta only for checking can be removed and fix the return value of the function later on to None
    # deltha_theta = learning_utils.Fed_avg_models(local_models, global_model, rounds_choise)
    # val_acc, val_loss = learning_utils.test(test_loader, global_model, test_criterion, args)
    # val_acc_list.append(val_acc) ; val_losses_list.append(val_loss)
    

    # # boardio.add_scalars("Losses over time in seconds", {"train_loss":avg_loss_over_chosen_users_curr_global_epoch,
    # #                                     "val_loss": val_loss}, time.time()-start_time)
    # # boardio.add_scalar('Val Accuracy', val_acc, time.time()-start_time)


    # time_counter += max_delay
    # print((f"global epoch {global_epoch} has been done artifficialy in {max_delay:.2f} secs, the total time by now is {time_counter:.2f} \n with avg train loss {avg_loss_over_chosen_users_curr_global_epoch:.3f}, avg val loss {val_loss:.3f}, avg val acc {val_acc:.2f}%"))
    # global_epochs_time_list.append(time_counter)
    # gc.collect()


    # if val_acc > best_val_acc:
    #     best_val_acc = val_acc
    #     torch.save({"model's state dict":global_model.state_dict(),
    #                 "train_loss_list": train_loss_list,
    #                 "val_acc_list": val_acc_list,
    #                 "val_losses_list": val_losses_list,
    #                 "global_epochs_time_list": global_epochs_time_list,
    #                 "num_of_users": args.num_users,
    #                 "num_of_users_per_round": args.num_users_per_round}
    #                 , path_best_model)
    
    
    # torch.save({"model's state dict":global_model.state_dict(),
    #             "train_loss_list": train_loss_list,
    #             "val_acc_list": val_acc_list,
    #             "val_losses_list": val_losses_list,
    #             "global_epochs_time_list": global_epochs_time_list,
    #             "num_of_obs_arr": num_of_obs_arr.reshape(-1),
    #             "global_epoch": global_epoch,
    #             "num_of_users": args.num_users,
    #             "num_of_users_per_round": args.num_users_per_round}
    #             , last_model_path)

    # if time_counter > args.max_seconds:
    #     users_idxs = tuple([str(x) for x in range(1,args.num_users+1)])
    #     fig, ax = plt.subplots()
    #     ax.bar(users_idxs, num_of_obs_arr.reshape(-1), width = 0.4)
    #     ax.set_title("Number of times each user was chosen")
    #     ax.set_ylabel("number of times")
    #     ax.set_xlabel("user index")
    #     #boardio.add_figure("Number of times each user was chosen", fig, global_epoch)
    #     plt.savefig(last_model_path.parent / "Number of times each user was chosen.png")
    #     break


#choices_table = choices_table.cumsum(axis=0)


  0%|          | 0/200 [00:00<?, ?it/s]

iteration1


  0%|          | 1/200 [00:00<02:15,  1.46it/s]

user 0, g: 0.027777777777777776,curr_delay = 0.07049809529445017, ucb: inf num_of_obs: 1
user 1, g: 0.027777777777777776,curr_delay = 0.056791396782105194, ucb: inf num_of_obs: 1
user 2, g: 0.027777777777777776,curr_delay = 0.06643864588826198, ucb: inf num_of_obs: 1
user 3, g: 0.027777777777777776,curr_delay = 0.10069492243540534, ucb: inf num_of_obs: 1
user 4, g: 0.027777777777777776,curr_delay = 0.09479443698512131, ucb: inf num_of_obs: 1
iteration2


  1%|          | 2/200 [00:01<02:35,  1.27it/s]

...

user 9, g: -2.550760126517707e-05,curr_delay = 0.13498482742748938, ucb: 2.492266204656199 num_of_obs: 35
user 13, g: 2.5507601265176792e-05,curr_delay = 0.1901370162126235, ucb: 2.493375583021933 num_of_obs: 33
user 15, g: 0.0009182736455463729,curr_delay = 0.7061125995746852, ucb: 2.494314146656628 num_of_obs: 28
user 16, g: 0.0009182736455463729,curr_delay = 0.7271905409084538, ucb: 2.493098640643545 num_of_obs: 28
user 17, g: 0.0009182736455463729,curr_delay = 0.763397004640805, ucb: 2.4914711071808897 num_of_obs: 28
iteration199


100%|█████████▉| 199/200 [02:38<00:00,  1.21it/s]

user 7, g: -0.0002027165419504001,curr_delay = 0.11687383796053447, ucb: 2.492104095984515 num_of_obs: 37
user 18, g: 0.0009602731693082938,curr_delay = 0.7422695883532477, ucb: 2.4913882078417626 num_of_obs: 28
user 19, g: 0.0009602731693082938,curr_delay = 0.7605412355277547, ucb: 2.4903517194995675 num_of_obs: 28
user 20, g: 0.0009602731693082938,curr_delay = 0.7535702920178614, ucb: 2.489212446786178 num_of_obs: 28
user 21, g: 0.0009602731693082938,curr_delay = 0.8012542607155216, ucb: 2.488326981829361 num_of_obs: 28
iteration200


100%|██████████| 200/200 [02:39<00:00,  1.26it/s]

user 11, g: 2.777777777777721e-06,curr_delay = 0.18998288137163954, ucb: 2.487294888948702 num_of_obs: 34
user 22, g: 0.0010027777777777767,curr_delay = 0.819963459416196, ucb: 2.488223773932637 num_of_obs: 28
user 23, g: 0.0010027777777777767,curr_delay = 0.8442835521176941, ucb: 2.4872422974948156 num_of_obs: 28
user 24, g: 0.0010027777777777767,curr_delay = 0.8436471070783517, ucb: 2.4862756416013836 num_of_obs: 28
user 25, g: 0.0010027777777777767,curr_delay = 0.8513115015999589, ucb: 2.4854959556338128 num_of_obs: 28


In [ ]:
path_dan = r"C:\dev\FL_privacy_mab\checkpoints\BSFL brute\cnn2\11-12-2023_18-27-08\last_model.pth.tar"
path_random = r"C:\dev\FL_privacy_mab\checkpoints\random selection\cnn2\11-12-2023_18-37-17\last_model.pth.tar"

random_dict = torch.load(path_random)
dan_dict = torch.load(path_dan)

random_train_loss_list = random_dict["train_loss_list"]
random_val_acc_list = random_dict["val_acc_list"]
random_val_losses_list = random_dict["val_losses_list"]
random_global_epochs_time_list = random_dict["global_epochs_time_list"]

dan_train_loss_list = dan_dict["train_loss_list"]
dan_val_acc_list = dan_dict["val_acc_list"]
dan_val_losses_list = dan_dict["val_losses_list"]
dan_global_epochs_time_list = dan_dict["global_epochs_time_list"]

fig, ax = plt.subplots(2,2, figsize=(15,15))

ax[0,0].plot(random_global_epochs_time_list , random_train_loss_list, label="train loss random selection", color = "blue")
ax[0,0].plot(dan_global_epochs_time_list, dan_train_loss_list, label="train loss dan's selection", color = "green")
ax[0,0].set_title("train loss over time")
ax[0,0].set_xlabel("time(sec)")
ax[0,0].set_ylabel("train loss")
ax[0,0].legend()

ax[0,1].plot(random_global_epochs_time_list , random_val_losses_list, label="val loss random selection", color = "blue")
ax[0,1].plot(dan_global_epochs_time_list, dan_val_losses_list, label="val loss dan's selection", color = "green")
ax[0,1].set_title("val loss over time")
ax[0,1].set_xlabel("time(sec)")
ax[0,1].set_ylabel("val loss")
ax[0,1].legend()

ax[1,0].plot(random_global_epochs_time_list , random_val_acc_list, label="val acc random selection", color = "blue")
ax[1,0].plot(dan_global_epochs_time_list, dan_val_acc_list, label="val acc dan's selection", color = "green")
ax[1,0].set_title("val acc over time")
ax[1,0].set_xlabel("time(sec)")
ax[1,0].set_ylabel("val acc")
ax[1,0].legend()



In [ ]:
%tensorboard --logdir=runs